In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from collections import Counter
import time
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
df = pd.read_csv("../input/train.tsv", delimiter="\t")

In [ ]:
df["name"] = df["name"].str.lower()
df["category_name"] = df["category_name"].str.lower()
df["brand_name"] = df["brand_name"].str.lower()
df["item_description"] = df["item_description"].str.lower()

In [ ]:
df["name"] = df["name"].str.replace(',', '')
df["name"] = df["name"].str.replace('.', '')

In [ ]:
from sklearn import preprocessing

In [ ]:
le = preprocessing.LabelEncoder()

In [ ]:
X = df.select_dtypes(include=[object])
X.head(3)

In [ ]:
X.isnull().sum()

In [ ]:
X["category_name"] = X["category_name"].fillna("-1")
X["brand_name"] = X["brand_name"].fillna("-1")
X = X.dropna()

In [ ]:
df_enc = X.apply(le.fit_transform)

df_enc.head(3)

In [ ]:
new_df = df

In [ ]:
new_df["name"] = df_enc["name"]
new_df["category_name"] = df_enc["category_name"]
new_df["brand_name"] = df_enc["brand_name"]
new_df["item_description"] = df_enc["item_description"]

In [ ]:
new_df.isnull().sum()

In [ ]:
new_df = new_df.dropna()
new_df.isnull().sum()

In [ ]:
price = new_df["price"].values

In [ ]:
new_df = new_df.drop("price", axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(new_df.values, price)

In [ ]:
from sklearn import linear_model

In [ ]:
clf = linear_model.Lasso()

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
pre = clf.predict(X_test)
pre

In [ ]:
def rmsle(pre, act):
    assert len(pre) == len(y_test), "different size of the array {0},{1}".format(len(pre),len(y_test))
    return np.sqrt(np.mean((np.log(pre+1)-np.log(act+1))**2.0))

In [ ]:
rmsle(pre, y_test)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
params = {
    "alpha":np.arange(0.1,1,0.05),
    "normalize": [True, False]
}

In [ ]:
grid_search = GridSearchCV(clf, param_grid=params)

In [ ]:
start = time.time()
grid_search.fit(X_train, y_train)
print("time", time.time()-start)

In [ ]:
pred = grid_search.predict(X_test)

In [ ]:
rmsle(pred, y_test)

In [ ]:
test_df = pd.read_csv("../input/test.tsv", delimiter="\t")
test_df.head(3)

In [ ]:
test_df["name"] = test_df["name"].str.lower()
test_df["category_name"] = test_df["category_name"].str.lower()
test_df["brand_name"] = test_df["brand_name"].str.lower()
test_df["item_description"] = test_df["item_description"].str.lower()

In [ ]:
test_df["name"] = test_df["name"].str.replace(',', '')
test_df["name"] = test_df["name"].str.replace('.', '')

In [ ]:
test_df.isnull().sum()

In [ ]:
test_X = test_df.select_dtypes(include=[object])
test_X.head(3)

In [ ]:
test_X["category_name"] = test_X["category_name"].fillna("-1")
test_X["brand_name"] = test_X["brand_name"].fillna("-1")
test_X = test_X.dropna()

In [ ]:
test_df_enc = test_X.apply(le.fit_transform)

test_df_enc.head(3)

In [ ]:
test_df["name"] = test_df_enc["name"]
test_df["category_name"] = test_df_enc["category_name"]
test_df["brand_name"] = test_df_enc["brand_name"]
test_df["item_description"] = test_df_enc["item_description"]

In [ ]:
test_df.head()

In [ ]:
test_pre = grid_search.predict(test_df)

In [ ]:
test_pre

In [ ]:
test = pd.DataFrame()

In [ ]:
test["test_id"] = np.arange(len(test_pre))

In [ ]:
test["price"] = test_pre

In [ ]:
test.to_csv("sub1.csv",index=False)